# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("proj").getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
from pyspark.sql.functions import corr

In [7]:
data.select(corr('Total_Purchase', 'Churn')).show()

+---------------------------+
|corr(Total_Purchase, Churn)|
+---------------------------+
|       0.024030894518220495|
+---------------------------+



In [9]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [13]:
indexer_name = StringIndexer(inputCol="Names", outputCol='name_index')
indexer_company = StringIndexer(inputCol="Company", outputCol = 'company_index')

In [14]:
data = indexer_name.fit(data).transform(data)

In [15]:
data = indexer_company.fit(data).transform(data)

In [16]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----------+-------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|name_index|company_index|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----------+-------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|     629.0|        824.0|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|     717.0|          1.0|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|     360.0|        272.0

In [17]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'name_index',
 'company_index']

In [18]:
assembler_1 = VectorAssembler(inputCols=['Age','Total_Purchase','Years', 'Num_Sites','name_index', 'company_index'],
                             outputCol='features')
assembler_2 = VectorAssembler(inputCols=['Age','Total_Purchase','Years', 'Num_Sites','company_index'], 
                              outputCol='features')

In [19]:
output1 = assembler_1.transform(data)
output2 = assembler_2.transform(data)

In [21]:
output1.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- name_index: double (nullable = true)
 |-- company_index: double (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
output2.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- name_index: double (nullable = true)
 |-- company_index: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
final_data1 = output1.select(['Churn', 'features'])
final_data2 = output2.select(['Churn', 'features'])

In [24]:
final_data2.show()

+-----+--------------------+
|Churn|            features|
+-----+--------------------+
|    1|[42.0,11066.8,7.2...|
|    1|[41.0,11916.22,6....|
|    1|[38.0,12884.75,6....|
|    1|[42.0,8010.76,6.7...|
|    1|[37.0,9191.58,5.5...|
|    1|[48.0,10356.02,5....|
|    1|[44.0,11331.58,5....|
|    1|[32.0,9885.12,6.9...|
|    1|[43.0,14062.6,5.4...|
|    1|[40.0,8066.94,7.1...|
|    1|[30.0,11575.37,5....|
|    1|[45.0,8771.02,6.6...|
|    1|[45.0,8988.67,4.8...|
|    1|[40.0,8283.32,5.1...|
|    1|[41.0,6569.87,4.3...|
|    1|[38.0,10494.82,6....|
|    1|[45.0,8213.41,7.3...|
|    1|[43.0,11226.88,8....|
|    1|[53.0,5515.09,6.8...|
|    1|[46.0,8046.4,5.69...|
+-----+--------------------+
only showing top 20 rows



In [25]:
train1, test1 = final_data1.randomSplit([0.7,0.3])
train2, test2 = final_data2.randomSplit([0.7,0.3])

In [26]:
train1.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                636|
|   mean| 0.1619496855345912|
| stddev|0.36869461658547925|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [27]:
test1.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                264|
|   mean|0.17803030303030304|
| stddev| 0.3832648210214543|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [28]:
from pyspark.ml.classification import LogisticRegression

In [31]:
logreg_model = LogisticRegression(labelCol='Churn')

In [32]:
lg_model1 = logreg_model.fit(train1)

In [33]:
lg_model2 = logreg_model.fit(train2)

In [34]:
fit_final1 = lg_model1.evaluate(test1)
fit_final2 = lg_model2.evaluate(test2)

In [38]:
fit_final1.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[26.0,8939.61,4.5...|[7.39386622139898...|[0.99938536443677...|       0.0|
|    0|[28.0,8670.98,3.9...|[9.26729628798032...|[0.99990554535134...|       0.0|
|    0|[28.0,9090.43,5.7...|[2.40513933138587...|[0.91721836461883...|       0.0|
|    0|[28.0,11204.23,3....|[2.02515209008249...|[0.88341269622541...|       0.0|
|    0|[29.0,9378.24,4.9...|[5.51746842344669...|[0.99600006185901...|       0.0|
|    0|[29.0,12711.15,5....|[6.37534050284828...|[0.99829985560642...|       0.0|
|    0|[30.0,8677.28,7.3...|[5.14336934232836...|[0.99419589790514...|       0.0|
|    0|[30.0,10183.98,5....|[3.58351116021080...|[0.97297276843107...|       0.0|
|    0|[30.0,12788.37,4....|[2.36780705653420...|[0.91433925869092...|       0.0|
|    0|[31.0,538

In [39]:
fit_final2.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[25.0,9672.03,5.4...|[4.51061370539412...|[0.98912779175429...|       0.0|
|    0|[26.0,8939.61,4.5...|[6.35295615855974...|[0.99826143676670...|       0.0|
|    0|[28.0,9090.43,5.7...|[1.87164083726380...|[0.86664802187224...|       0.0|
|    0|[28.0,11204.23,3....|[2.09320284271890...|[0.89024077362861...|       0.0|
|    0|[29.0,13240.01,4....|[7.39385857155902...|[0.99938535973778...|       0.0|
|    0|[30.0,8677.28,7.3...|[4.20470875218061...|[0.98529435140321...|       0.0|
|    0|[31.0,8829.83,4.5...|[4.87011148200159...|[0.99238590929282...|       0.0|
|    0|[31.0,9574.89,7.3...|[2.94087368635796...|[0.94983037663063...|       0.0|
|    0|[32.0,5756.12,5.9...|[3.98963752133596...|[0.98182984348078...|       0.0|
|    0|[32.0,857

In [40]:
fit_final1.areaUnderROC

0.8391508971467795

In [41]:
fit_final2.areaUnderROC

0.8948497854077255

In [43]:
fit_final1.precisionByThreshold.show()

+------------------+------------------+
|         threshold|         precision|
+------------------+------------------+
|0.9912206977768123|               1.0|
|0.9682866450708456|               1.0|
|0.9464910499174567|               1.0|
|0.9082687860425829|               1.0|
| 0.891517980672415|               1.0|
|0.8825631897982636|0.9166666666666666|
|0.8093207602587302|0.8571428571428571|
|0.7958430487331845|            0.8125|
| 0.771205782721698|0.7777777777777778|
|0.7586558645506458|              0.75|
|0.7431504801228754|0.7727272727272727|
|0.7247106193342866|0.7083333333333334|
| 0.669006245397715|0.6923076923076923|
|0.6398407327007147|0.7142857142857143|
| 0.615514991043939|0.6666666666666666|
| 0.558691367228683|           0.65625|
|0.5503378906328471|0.6764705882352942|
|0.5479044292594594|0.6388888888888888|
|0.4774169432123496|0.6052631578947368|
|0.4659809949638232|             0.575|
+------------------+------------------+
only showing top 20 rows



In [44]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [48]:
my_eval = BinaryClassificationEvaluator(labelCol='Churn')

In [49]:
my_final_roc1 = my_eval.evaluate(fit_final1.predictions)

In [50]:
my_final_roc2 = my_eval.evaluate(fit_final2.predictions)

In [51]:
my_final_roc1

0.839396019217571

In [52]:
my_final_roc2

0.8947672499174638

### Predict on new data

In [53]:
new_data = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [55]:
new_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [56]:
new_data = indexer_company.fit(new_data).transform(new_data)

In [57]:
new_output_data = assembler_2.transform(new_data)

In [58]:
final_new_data = new_output_data.select(['features'])

In [61]:
final_result = lg_model2.transform(new_output_data)

In [62]:
final_result.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|company_index|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|          4.0|[37.0,9935.53,7.7...|[2.21923021410118...|[0.90196314796023...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|          3.0|

In [63]:
final_result.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

